In [0]:
import pandas as pd
import numpy as np
import json

In [0]:
file_path = "https://raw.githubusercontent.com/VieEeEw/CS296-project1/master/data.csv"

In [0]:
rawDF = pd.read_csv(file_path, low_memory=False)
rawDF = rawDF.fillna(0)[['Fall', 'College', 'Major Name', 'Total']]
rawDF['Total'] = pd.to_numeric(rawDF['Total']
                               .astype(str)
                               .replace(",","", regex=True)
                               .loc[rawDF['Total'] != 'Art and Design'])

procDF = (rawDF
          .loc[rawDF['Major Name'].str.isupper() == False]
         .groupby(['Fall', 'College', 'Major Name'], as_index=False)
          .sum())

outDF = (procDF
         .set_index(["Fall", "College", 'Major Name'])
         .unstack(level=0)
         .fillna(0)
         .stack()
         .reset_index()
        )


test = outDF.loc[outDF['College'] == "Engineering"][['Major Name', 'Fall', 'Total']]

totals = outDF.groupby(['College', 'Fall'])['Total'].sum().reset_index()

totals['Major Name'] = 'Total'

finalDF = pd.concat([outDF, totals], sort=False).sort_values(['College','Major Name', 'Fall', 'Total'])


json_out = (finalDF.groupby(['College', 'Major Name'])
    .apply(lambda x: x[['Fall', 'Total']].to_dict('records'))
    .reset_index()
    .rename(columns={0:'Count'})
    .groupby(['College'])
    .apply(lambda x: dict(zip(x['Major Name'], x['Count'])))
    .reset_index()
    .rename(columns={0:'Major'})
    .to_json(orient='records')
)


json.loads(json_out)

# with open('out.json', 'w') as file:
#   json.dump(json_out, file)
  

[{'College': 'ACES',
  'Major': {'Agr & Consumer Economics': [{'Fall': 1980, 'Total': 0.0},
    {'Fall': 1981, 'Total': 0.0},
    {'Fall': 1982, 'Total': 0.0},
    {'Fall': 1983, 'Total': 0.0},
    {'Fall': 1984, 'Total': 0.0},
    {'Fall': 1985, 'Total': 0.0},
    {'Fall': 1986, 'Total': 0.0},
    {'Fall': 1987, 'Total': 0.0},
    {'Fall': 1988, 'Total': 0.0},
    {'Fall': 1989, 'Total': 0.0},
    {'Fall': 1990, 'Total': 0.0},
    {'Fall': 1991, 'Total': 0.0},
    {'Fall': 1992, 'Total': 0.0},
    {'Fall': 1993, 'Total': 0.0},
    {'Fall': 1994, 'Total': 0.0},
    {'Fall': 1995, 'Total': 0.0},
    {'Fall': 1996, 'Total': 0.0},
    {'Fall': 1997, 'Total': 181.0},
    {'Fall': 1998, 'Total': 259.0},
    {'Fall': 1999, 'Total': 318.0},
    {'Fall': 2000, 'Total': 316.0},
    {'Fall': 2001, 'Total': 326.0},
    {'Fall': 2002, 'Total': 352.0},
    {'Fall': 2003, 'Total': 360.0},
    {'Fall': 2004, 'Total': 85.0},
    {'Fall': 2005, 'Total': 83.0},
    {'Fall': 2006, 'Total': 336.0},
    {'